# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/987.6 kB ? eta -:--:--
     - -------------------------------------- 41.0/987.6 kB ? eta -:--:--
     ----- -------------------------------- 143.4/987.6 kB 1.7 MB/s eta 0:00:01
     ----- -------------------------------- 153.6/987.6 kB 1.5 MB/s eta 0:00:01
     --------- ---------------------------- 235.5/987.6 kB 1.3 MB/s eta 0:00:01
     --------- ---------------------------- 235.5/987.6 kB 1.3 MB/s eta 0:00:01
     --------------- ---------------------- 399.4/987.6 kB 1.4 MB/s eta 0:00:01
     ------------------ ------------------- 481.3/987.6 kB 1.4 MB/s eta 0:00:01
     -------------------- ----------------- 532.5/987.6 kB 1.4 MB/s eta 0:00:01
     ----------------------- -------------- 614.4/987.6 kB 1.4 MB/s eta 0:00:01
     -------------------------- ----------- 686.1/987.6 kB 1.4 MB/s eta 0:00:01
     -------------------------- ----------- 696.3/987.6 kB 1.4 MB/s eta 

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [16]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [30]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位很棒的生物学家。你擅长回答生物学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

computer_template = """你是一位很棒的计算机专家。你擅长回答计算机问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

chinese_language_template = """你是一位很棒的汉语言文学家。你擅长回答汉语言文学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


In [31]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物学",
        "description": "适用于回答生物学问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_language_template,
    },
]

In [32]:
llm = OpenAI(
    # 代理地址，填写商家中转站或自建OpenAI代理
    base_url='https://api.xiaoai.plus/v1',
    model_name="gpt-3.5-turbo-instruct"
)

In [33]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [34]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [35]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [36]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [37]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物学: 适用于回答生物学问题', '计算机: 适用于回答计算机问题', '汉语言文学: 适用于回答汉语言文学问题']


In [38]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物学: 适用于回答生物学问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题


In [39]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [40]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物学: 适用于回答生物学问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}


In [41]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [42]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？'}
> Finished chain.
{'input': '黑体辐射是什么？', 'text': '\n\n黑体辐射指的是温度为绝对零度时，理想的完美辐射体所发出的电磁波。它的光谱分布与其温度有关，符合普朗克定律。这种辐射在物理学和天文学中具有重要意义，可以用来解释物体的发光和吸收现象。'}


In [43]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n首先，我们需要知道什么是质数。质数是指只能被1和自身整除的正整数，例如2、3、5、7等。根据这个定义，我们可以列出大于40的前几个质数：41、43、47。\n\n接下来，我们需要找到一个符合条件的质数，使得它加一能被3整除。我们可以尝试将41、43、47分别加一，发现只有47加一后能被3整除，即48除以3等于16。\n\n因此，大于40的第一个质数是47，使得这个质数加一能被3整除。'}


In [44]:
router_chain.verbose = True

In [45]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n\n黑洞是宇宙中极其密集的天体，它的引力非常强大，甚至连光也无法逃脱。它的引力来自于它非常大的质量，因为它的体积很小，所以它的密度也非常高。由于黑洞具有强大的引力，它会吸引附近的物质，形成一个巨大的旋涡，这也是我们能够发现黑洞的一种方式。黑洞可以吞噬掉附近的星系和星云，因此被称为宇宙中的“吞噬者”。虽然我们无法直接观测到黑洞，但通过观测它们对周围物质的影响，我们可以推断它们的存在。黑洞是宇宙中非常神秘的天体，它们'}


In [46]:
print(chain.invoke("Java和C语言，哪个是面向对象的？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': 'Java和C语言，哪个是面向对象的？'}
> Finished chain.
{'input': 'Java和C语言，哪个是面向对象的？', 'text': '\n\nJava是面向对象的语言，而C语言是面向过程的语言。面向对象的语言具有更强的封装性、继承性和多态性，可以更好地组织和管理程序的结构，提高代码的复用性和可维护性。相比之下，C语言更偏向于面向过程的编程，关注的是算法和步骤的实现。'}


In [47]:
print(chain.invoke("肠胃炎的常见病因是什么"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物学: {'input': '肠胃炎的常见病因是什么'}
> Finished chain.
{'input': '肠胃炎的常见病因是什么', 'text': '？\n\n首先，肠胃炎是指胃肠道发炎引起的症状，通常由病毒、细菌或其他微生物感染引起。因此，肠胃炎的常见病因就是病原体的感染。\n\n其次，肠胃炎的病原体可以是多种多样的，最常见的是病毒感染，如轮状病毒、腺病毒等，也有可能是细菌感染，如沙门氏菌、大肠杆菌等。此外，寄生虫和真菌也可能引起肠胃炎。\n\n最后，除了病原体感染外，肠胃炎的发病也可能与不良的饮食习惯、饮用不洁水源、接触污'}


In [48]:
print(chain.invoke("为什么小说很喜欢用比喻手法？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学: {'input': '小说很喜欢用比喻手法?'}
> Finished chain.
{'input': '小说很喜欢用比喻手法?', 'text': '\n是的，小说中常常使用比喻手法来丰富文学意境和表达深层含义。比喻是一种修辞手法，通过将一个事物与另一个事物进行比较，来传达出更深层次的含义。它可以让读者更容易理解作者想要表达的情感和思想，也能够增加作品的艺术性。比喻的使用也可以让作品更具有感染力和生动性，使人更容易产生共鸣。因此，小说中使用比喻手法是非常常见的现象。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

### RouterChain 的理解与分析

1. destination_chains中，每一个Chain，由LLM、Prompt和OutputParser
   1. LLM用于解析Prompt，并返回结果，最终结果由OutputParser解析成字典
   2. 默认OutputParser返回的是字典，包含变量（这里有一个input变量）和text
2. router_chain，也需要由LLM、Prompt和OutputParser组成
   1. 不同的是RoutrChain提供了PromptTemplate，我们只需要给定CANDIDATE PROMPTS（包含提示符和对应描述，描述每个destination_chains的作用含义等），
   2. RoutrChain的PromptTemplate会使得LLM返回JSON格式的数据，JSON中包含提示符和input
   3. OutputParser会将JSON解析，将提示符作为字典搜索的KEY获取到对应的destination_chains的Chain
3. default_chain 用于router_chain中LLM的Response是LLM是DEFAULT时使用